## Inconsistent Date Check

In [ ]:
from datetime import date
def INCONSISTENT_DATE_CHECK(sources,df,measure):
    try:
        # df = pd.read_csv('Downloads/config3.csv')
        
        inconsistency_date_check_rows = df[(df['Function_Type']=='Inconsistent_Date_Check') & (df['DataSource'] == sources)]
        engine = create_engine(db_connection_string)

        result_list = []
        for index, row in inconsistency_date_check_rows.iterrows():
            test_case_id = row['TC_ID']
            
            data_source = row['DataSource']
            table_name = row['TableName']
            column_name = row['Column']
            base_table_member = row['BaseTableMember']
            check_data_source = row['checkdatasource']
            check_table = row['check_table']
            check_column = row['check_column']
            check_table_member = row['CheckTableMember']

            query = f"""
            WITH base_data AS (
                SELECT DISTINCT [{base_table_member}], [{column_name}] 
                FROM [{data_source}].[{table_name}]
            ),
            check_data AS (
                SELECT DISTINCT [{check_table_member}], [{check_column}]
                FROM [{check_data_source}].[{check_table}]
            ),
           
            inconsistent_members AS (
                SELECT b.[{base_table_member}]
                FROM base_data b
                INNER JOIN check_data c ON cast(b.[{base_table_member}] as nvarchar) = cast(c.[{check_table_member}] as nvarchar)
                WHERE cast(b.[{column_name}] as date) != cast(c.[{check_column}] as date)
            )
            SELECT 
                (SELECT COUNT(DISTINCT [{base_table_member}]) FROM base_data),
                (SELECT COUNT(DISTINCT [{base_table_member}]) FROM inconsistent_members)
            """
            
            query2 = f"""
            WITH base_data AS (
                SELECT DISTINCT [{base_table_member}], [{column_name}] 
                FROM [{data_source}].[{table_name}]
            ),
            check_data AS (
                SELECT DISTINCT [{check_table_member}], [{check_column}]
                FROM [{check_data_source}].[{check_table}]
            ),
           
            inconsistent_members AS (
                SELECT b.[{base_table_member}],b.[{column_name}] as primary_date, c.[{check_column}] as comparing_Date
                FROM base_data b
                INNER JOIN check_data c ON cast(b.[{base_table_member}] as nvarchar) = cast(c.[{check_table_member}] as nvarchar)
                WHERE cast(b.[{column_name}] as date) != cast(c.[{check_column}] as date)
            )
            SELECT * from inconsistent_members
                
            """




            
            with engine.connect() as conn:
                result = conn.execute(text(query)).fetchone()
                result2 = conn.execute(text(query2)).fetchall()
            # print(result)
            # print(result2)
            total_members, inconsistent_members = result
            invalid_member_data = []
            invalid_rows = 0
            for row in result2:
                invalid_rows += 1
                invalid_member_data.append({
                        'MemberId': row[0],
                        'Primary Date': row[1].strftime('%m/%d/%Y') if row[1] else None,
                        'Comparing Date': row[2].strftime('%m/%d/%Y') if row[2] else None,
                        'Error Flag' : 1
            })

            

            
            member_percentage = (inconsistent_members / total_members) * 100 if total_members > 0 else 0
            error_flag = 1 if inconsistent_members > 0 else 0 

            result_list.append({
                'DVE Run Date' : date.today().strftime('%m%d%Y'),
                'TestCaseID': test_case_id,
                'Measure':measure,
                'Function' : "Inconsistent Date Check",
                "Comparision" : "% of values with inconsistent date",
                'DataSource': data_source,
                'TableName': table_name,
                'ColumnName': column_name,
                'Check Data Source' : check_data_source,
                'Check Table Name' : check_table,
                'Check Column Name' :check_column,
                'Total Unique Member IDs': total_members,
                'Inconsistent Date Member IDs': inconsistent_members,
                'Member Percentage': f"{member_percentage:.2f}%",
                'Invalid Member Data' : invalid_member_data,
                'Error Flag': error_flag
            })

        result_json = json.dumps(result_list)
        return result_list

    except FileNotFoundError:
        return json.dumps({"error": "Excel file not found."})
    except SQLAlchemyError as e:
        return json.dumps({"error": f"Database error: {str(e)}"})
    except Exception as e:
        return json.dumps({"error": f"An error occurred: {str(e)}"})
db_connection_string = 'mssql+pyodbc://zsaudit:9v]3>SYv1C20@lhp-db-copy.database.windows.net:1433/HEDIS_2023_Copy?driver=ODBC+Driver+17+for+SQL+Server'
# b = INCONSISTENT_DATE_CHECK('LWCC')
# b

## Date Format Check

In [ ]:
from datetime import date
def DATE_FORMAT_CHECK(sources,df,measure):
    try:
        # df = pd.read_csv('Downloads/config3.csv')
        date_format_check_rows = df[(df['Function_Type'] == 'Date_Format_Check') & (df['DataSource'] == sources)]
        engine = create_engine(db_connection_string)
        print("engine created")
        result_list = []
        today = date.today()
        for index, row in date_format_check_rows.iterrows():
            test_case_id = row['TC_ID']
            data_source = row['DataSource']
            table_name = row['TableName']
            base_table_member = row['BaseTableMember']
            column_name = row['Column']
            print(column_name)
            # Modified query to get member ids and dates with incorrect format or 1/1/1900
            query = f"""
                SELECT
                    {column_name},
                    [{base_table_member}]
                    
                FROM {data_source}.{table_name}
                WHERE
                    {column_name} is NULL
                    OR TRY_CONVERT(DATE, {column_name}) IS NULL
                    OR {column_name} = '1900-01-01'  
            """

            with engine.connect() as conn:
                result = conn.execute(text(query)).fetchall()  # Fetch all rows

                total_rows = 0
                invalid_rows = 0
                invalid_member_data = []  # Store member ids and dates

                for row in result:
                    total_rows += 1
                    invalid_rows += 1
                    invalid_member_data.append({
                        'MemberId': row[1],
                        'Date': row[0].strftime('%m/%d/%Y') if row[0] else None,
                        'Error' : 1
                    })

                if total_rows > 0:
                    invalid_percentage = (invalid_rows / total_rows) * 100
                else:
                    invalid_percentage = 0

                error_flag = 1 if invalid_rows > 0 else 0
                result_list.append({
                    'DVE_Run_Date' : str(today),
                    'TestCaseID': test_case_id,
                    'Measure' : measure,
                    'Function' : "Date Format Check",
                    'Comparision' : '% of dates with incorrect format',
                    'DataSource': data_source,
                    'TableName': table_name,
                    'ColumnName': column_name,
                    'Total rows': total_rows,
                    'Invalid Rows': invalid_rows,
                    'Percentage': f"{invalid_percentage:.2f}%",
                    'Error Flag': error_flag,
                    'InvalidMemberData': invalid_member_data
                })

        result_json = json.dumps(result_list)
        return result_list

    except FileNotFoundError:
        return json.dumps({"error": "Excel file not found."})
    except SQLAlchemyError as e:
        return json.dumps({"error": f"Database error: {str(e)}"})
    except Exception as e:
        return json.dumps({"error": f"An error occurred: {str(e)}"})



# db_connection_string = 'mssql+pyodbc://zsaudit:9v]3>SYv1C20@lhp-db-copy.database.windows.net:1433/HEDIS_2023_Copy?driver=ODBC+Driver+17+for+SQL+Server'
# print(DATE_FORMAT_CHECK('LWCC'))

## Age Check

In [ ]:
def AGE_CHECK_pre(sources,table_name,columnname, threshold = 0.1,previous_results = {}):
    try:
        engine = create_engine(db_connection_string)
        today = date.today()
            # print(current_counts)
        # today = datetime.now().strftime('%m%d%Y')
        query = f"""
                SELECT age_group,count(distinct memberID) as members from
                (select memberID,
                    CASE
                        WHEN DATEDIFF(YEAR, {columnname}, cast(GETDATE() as date)) BETWEEN 0 AND 5 THEN '0-5'
                        WHEN DATEDIFF(YEAR, {columnname}, cast(GETDATE() as date)) BETWEEN 6 AND 10 THEN '6-10'
                        WHEN DATEDIFF(YEAR, {columnname}, cast(GETDATE() as date)) BETWEEN 11 AND 15 THEN '11-15'
                        WHEN DATEDIFF(YEAR, {columnname}, cast(GETDATE() as date)) BETWEEN 16 AND 20 THEN '16-20'
                        WHEN DATEDIFF(YEAR, {columnname}, cast(GETDATE() as date)) BETWEEN 21 AND 25 THEN '21-25'
                        WHEN DATEDIFF(YEAR, {columnname}, cast(GETDATE() as date)) BETWEEN 26 AND 30 THEN '26-30'
                        WHEN DATEDIFF(YEAR, {columnname}, cast(GETDATE() as date)) BETWEEN 31 AND 35 THEN '31-35'
                        WHEN DATEDIFF(YEAR, {columnname}, cast(GETDATE() as date)) BETWEEN 36 AND 40 THEN '36-40'
                        WHEN DATEDIFF(YEAR, {columnname}, cast(GETDATE() as date)) BETWEEN 41 AND 45 THEN '41-45'
                        WHEN DATEDIFF(YEAR, {columnname}, cast(GETDATE() as date)) BETWEEN 46 AND 50 THEN '46-50'
                        WHEN DATEDIFF(YEAR, {columnname}, cast(GETDATE() as date)) BETWEEN 51 AND 55 THEN '51-55'
                        WHEN DATEDIFF(YEAR, {columnname}, cast(GETDATE() as date)) BETWEEN 56 AND 60 THEN '56-60'
                        
                        ELSE 'Other'
                    END AS age_group
                    FROM {sources}.{table_name}
                     )as subquery
                    
                GROUP BY age_group
                """
        with engine.connect() as conn:
            current_year_results = conn.execute(text(query)).fetchall()
        current_year_total = sum(row[1] for row in current_year_results)
        
        current_year_counts = {row[0]: row[1] for row in current_year_results}
        # for row in current_year_results:
        #     print(row)
        today = date.today().strftime('%m%d%Y')
        
        if today in previous_results:
            previous_counts = previous_results[today]
        else:
            if previous_results:
                latest_date = max(previous_results.keys())
                       
                previous_counts = previous_results[latest_date]
            else:
                previous_counts = current_year_counts.copy()
                
       
                
        error_flag = 0
        error_flag_list = []
        for category,current_count in current_year_counts.items():
            
            previous_count = previous_counts.get(category,0)
            # print(category,current_count,previous_count)
            if previous_count>0:
                percentage_change = abs((current_count - previous_count)/previous_count)
            else:
                percentage_change = current_count
            if percentage_change>threshold:
                error_flag = 1
                error_flag_list.append(1)
            else:
                error_flag_list.append(0)
        print(error_flag_list)
            
                      
        previous_results[today] = current_year_counts

                # print(current_year_counts)
                # print("--------")
                # print(previous_results)
        return current_year_counts,previous_results,error_flag,error_flag_list
    except FileNotFoundError:
        return json.dumps({"error": "Excel file not found."})
    except SQLAlchemyError as e:
        return json.dumps({"error": f"Database error: {str(e)}"})
    except Exception as e:
        return json.dumps({"error": f"An error occurred: {e}"})
        

In [ ]:
def AGE_CHECK_main(sources,final_list,df,measure):
    # measure_to_value_filtered = measure_to_value[measure_to_value['Measure ID']==measure]
    # df = pd.read_csv('config3.csv')
    age_check_rows = df[(df['Function_Type'] == 'Age_Check') & (df['DataSource'] == sources)]
    engine = create_engine(db_connection_string)
    value_set_final_list = []
    source_rows = age_check_rows[age_check_rows['DataSource'] == sources]
    for index, row in source_rows.iterrows():
        value_set_final_dict = {}
        test_case_id = row['TC_ID']
        table_name = row['TableName']
        columnname = row['Column']  
        
        # filtered_data = [d for d in final_list]
        result = None
        # for d in filtered_data:
        #     if code_system in d['Previous Count']:
        if len(final_list) !=0:
            result = final_list[0]
                
        if result is not None:  
            previous_results = {}
            date_prev = result['DVE Run Date']
            previous_results[date_prev] = result['Previous Count']
        else:
            previous_results = {}
        current_counts,previous_results,error_flag,error_flag_list = AGE_CHECK_pre(sources,table_name,columnname, threshold = 0.1,previous_results = {})
        value_set_final_dict['DVE Run Date'] = date.today().strftime('%m%d%Y')
        value_set_final_dict['Test Case ID'] = test_case_id
        value_set_final_dict['Measure'] = measure
        value_set_final_dict['Function'] = "Age Check"
        value_set_final_dict['Comparision'] = "% of members with inconsistent Age"
        value_set_final_dict['Datasource'] = sources
        value_set_final_dict['TableName'] = table_name
        value_set_final_dict['ColumnName'] = columnname
        value_set_final_dict['Current Count'] = current_counts
        value_set_final_dict['Previous Count'] = previous_results
        value_set_final_dict['Error Flag Individual'] = error_flag_list
        value_set_final_dict['Error Flag'] = error_flag
        value_set_final_list.append(value_set_final_dict)
    return value_set_final_list
        

        
    


## Future Date Check

In [ ]:
def FUTURE_DATE_CHECK(sources,df,measure):
    try:
        # df = pd.read_csv('config3.csv')
        future_date_check_rows = df[(df['Function_Type'] == 'Future_Date_Check') & (df['DataSource'] == sources)]
        engine = create_engine(db_connection_string)
        threshold_percentage = 0

        result_list = []

        for index, row in future_date_check_rows.iterrows():
            test_case_id = row['TC_ID']
            data_source = row['DataSource']
            table_name = row['TableName']
            column_name = row['Column']
            base_table_member = row['BaseTableMember']
            check_data_source = row['checkdatasource']
            check_table = row['check_table']
            check_column = row['check_column']
            check_table_member = row['CheckTableMember']

            query = f"""
            WITH base_data AS (
                SELECT   [{base_table_member}], max({column_name}) as column_name
                FROM [{data_source}].[{table_name}] group by [{base_table_member}]
            ),
            check_data AS (
                SELECT  [{check_table_member}], max({check_column}) as check_column
                FROM [{check_data_source}].[{check_table}] group by [{check_table_member}]
            ),
            inconsistent_members AS (
                SELECT distinct(b.[{base_table_member}]),b.column_name, c.check_column
                FROM base_data b
                JOIN check_data c ON cast(b.[{base_table_member}] as nvarchar) = cast(c.[{check_table_member}] as nvarchar)
                WHERE cast(b.column_name as date) < cast(c.check_column as date) 
                
            )
            SELECT * from inconsistent_members
                
            """
            query2 = f"""
            WITH base_data AS (
                SELECT DISTINCT  [{base_table_member}], {column_name}
                FROM [{data_source}].[{table_name}]
            ),
            check_data AS (
                SELECT DISTINCT [{check_table_member}], {check_column}
                FROM [{check_data_source}].[{check_table}]
            ),
            inconsistent_members AS (
                SELECT b.[{base_table_member}], b.{column_name}, c.{check_column}
                FROM base_data b
                JOIN check_data c ON cast(b.[{base_table_member}] as nvarchar) = cast(c.[{check_table_member}] as nvarchar)
                WHERE cast(b.{column_name} as date) < cast(c.{check_column} as date)
            )
            SELECT count(distinct [{base_table_member}]) from [{data_source}].[{table_name}]
                
            """
            with engine.connect() as conn:
                result = conn.execute(text(query)).fetchall()
                result2 = conn.execute(text(query2)).fetchone()# Fetch all rows
                
                total_rows = result2[0]
                invalid_rows = 0
                invalid_member_data = []  # Store member ids and dates

                for row in result:
                    # print(row[1])
                    # total_rows += 1
                    invalid_rows += 1
                    invalid_member_data.append({
                        'MemberId': row[0],
                        'Primary Date': row[1].strftime('%m/%d/%Y') if row[1] else None,
                        'Comparing Date': row[2].strftime('%m/%d/%Y') if row[2] else None,
                        'Error Flag' : 1
                    })

                if total_rows > 0:
                    invalid_percentage = (invalid_rows / total_rows) * 100
                else:
                    invalid_percentage = 0

                error_flag = 1 if invalid_rows > 0 else 0
                result_list.append({
                    'TestCaseID': test_case_id,
                    'Measure': measure,
                    'DataSource': data_source,
                    'Comparision': '% of dates in future',
                    'TableName': table_name,
                    'ColumnName': column_name,
                    'CheckTableName': check_table,
                    'CheckColumnName': check_column,
                    'Total Unique Member IDs': total_rows,
                    'Future Date Member IDs': invalid_rows,
                    'Member Percentage': f"{invalid_percentage:.2f}%",
                    'Error Flag': error_flag,
                    'Invalid Member Data': invalid_member_data
                })





        result_json = json.dumps(result_list)
        return result_list

    except FileNotFoundError:
        return json.dumps({"error": "Excel file not found."})
    except SQLAlchemyError as e:
        return json.dumps({"error": f"Database error: {str(e)}"})
    except Exception as e:
        return json.dumps({"error": f"An error occurred: {str(e)}"})
